# Model 1: Modeling Selected Actions

The purpose of this model is to predict the action that the bot will select for a given state of the simulation.

## Preliminary Setup

### Imports

In [1]:
import os
import numpy as np
import random
import math

In [2]:
# %pip install nbimporter

In [3]:
import nbimporter

### Additional Bot Config

In [4]:
import Bot1

In [5]:
%run Bot1.ipynb

In [6]:
# 1 crew 1 alien bot 
def Bot1(k, alpha, max_iter, timeout):
    grid, open_cells = create_grid()
    bot, ship, open_cells = place_bot(grid, open_cells)

    crew_list = []
    alien_list = []
    d_lookup_table = {}

    # Place 1 crew member + 1 alien
    crew_list, ship = place_crew(ship, open_cells, crew_list)
    alien_list, ship = place_alien(ship, open_cells, alien_list, bot, k)

    alien_matrix = initialize_alienmatrix(open_cells, bot, k)
    crew_matrix = initialize_crewmatrix(open_cells, crew_list, bot)

    win_count = 0
    loss_count = 0
    move = 0
    win_move_count = []
    marker = 0

    while (win_count + loss_count) < max_iter:
        neighbors = check_valid_neighbors(len(ship), bot[0], bot[1])
        open_moves = [neigh for neigh in neighbors if (grid[neigh] != 1)]
        open_moves.append(bot) # Bot can stay in place 
        next_move = determine_move(open_moves, alien_matrix, crew_matrix)
        
        prev_win_count = win_count
        bot, crew_list, ship, open_cells, win_count, marker = move_bot(ship, bot, next_move, crew_list, alien_list, open_cells, win_count, 1)
        move += 1

        if marker == 1 or move >= timeout:
            loss_count += 1
            print(f"Bot captured! Win Count: {win_count}, Loss Count: {loss_count}")

            grid, open_cells = reset_grid(grid, open_cells)
            bot, ship, open_cells = place_bot(grid, open_cells)
            crew_list = []
            alien_list = []
            d_lookup_table = {}

            crew_list, ship = place_crew(ship, open_cells, crew_list)
            alien_list, ship = place_alien(ship, open_cells, alien_list, bot, k)

            alien_matrix = initialize_alienmatrix(open_cells, bot, k)
            crew_matrix = initialize_crewmatrix(open_cells, crew_list, bot)
            marker = 0
            move = 0

            continue

        if win_count > prev_win_count:
            print(f"Crew saved! Win Count: {win_count}, Loss Count: {loss_count}")
            win_move_count.append(move)
            move = 0
            d_lookup_table = {}
            alien_matrix = initialize_alienmatrix(open_cells, bot, k)
            crew_matrix = initialize_crewmatrix(open_cells, crew_list, bot)
        
        print(f"Bot: {bot}, Crew: {crew_list}, Aliens: {alien_list}")

        alien_matrix, crew_matrix = update_afterbotmove(bot, alien_matrix, crew_matrix)

        # Move bot to optimal neighbor
        marker, alien_list, ship = move_aliens(ship, alien_list, bot) # Move alien randomly

        if marker == 1 or move >= timeout:
            loss_count += 1
            print(f"Bot captured! Win Count: {win_count}, Loss Count: {loss_count}")

            grid, open_cells = reset_grid(grid, open_cells)
            bot, ship, open_cells = place_bot(grid, open_cells)
            crew_list = []
            alien_list = []
            d_lookup_table = {}

            crew_list, ship = place_crew(ship, open_cells, crew_list)
            alien_list, ship = place_alien(ship, open_cells, alien_list, bot, k)

            alien_matrix = initialize_alienmatrix(open_cells, bot, k)
            crew_matrix = initialize_crewmatrix(open_cells, crew_list, bot)
            marker = 0
            move = 0

            continue
        
        alien_matrix = update_afteralienmove(ship, alien_list, alien_matrix) # Update after alien move
        
        alien_detected = alien_sensor(alien_list, bot, k) # Run Alien Sensor
        crew_detected, d_lookup_table = crew_sensor(ship, bot, alpha, d_lookup_table, crew_list) # Run Crew Sensor
        
        alien_matrix = update_alienmatrix(alien_matrix, alien_detected, bot, k) # Update based on alien sensor

        crew_matrix = update_crewmatrix(crew_matrix, crew_detected, d_lookup_table, bot, alpha) # Update based on crew sensor 

    return sum(win_move_count) // max(1, len(win_move_count)), (win_count / max(1, (win_count + loss_count))), win_count

In [7]:
def Bot1_simulation(alpha_values, k_values, max_iter, timeout, num_simulations=10):
    avg_rescue_moves = {k: [] for k in k_values}
    prob_crew_rescue = {k: [] for k in k_values}
    avg_crew_saved = {k: [] for k in k_values}

    for k in k_values:
        for alpha in alpha_values:
            total_metric1, total_metric2, total_metric3 = 0, 0, 0
            
            for i in range(num_simulations):
                metric1, metric2, metric3 = Bot1(k, alpha, max_iter, timeout)
                total_metric1 += metric1
                total_metric2 += metric2
                total_metric3 += metric3

            avg_metric1 = total_metric1 / num_simulations
            avg_metric2 = total_metric2 / num_simulations
            avg_metric3 = total_metric3 / num_simulations

            print(f"k: {k}, Alpha: {alpha}\nAverage Rescue Moves: {avg_metric1}\nProbability of Crew Rescue: {avg_metric2}\nAverage Crew Saved: {avg_metric3}\n")

            avg_rescue_moves[k].append(avg_metric1)
            prob_crew_rescue[k].append(avg_metric2)
            avg_crew_saved[k].append(avg_metric3)

    return avg_rescue_moves, prob_crew_rescue, avg_crew_saved

In [8]:
def plot_Bot1(alpha_values, k_values, bot1_data, title, metric_num):
    # Generate a plot for each k-value
    for k in k_values:
        plt.figure(figsize=(10, 6))
        plt.plot(alpha_values, bot1_data[k], label=f'Bot 1, k={k}')
        plt.title(f'{title} (k={k})')
        plt.xlabel('alpha')
        plt.ylabel(title)

        # Set x-axis ticks
        plt.xticks(alpha_values, labels=[str(alpha) for alpha in alpha_values])

        plt.legend()
        plt.grid(True)
        
        plt.show()

In [9]:
def one_alien_one_crew(alpha_values, k_values, max_iter, timeout):
    bot1_avg_rescue_moves, bot1_prob_crew_rescue, bot1_avg_crew_saved = Bot1_simulation(alpha_values, k_values, max_iter, timeout)

    bot1_prob_crew_rescue = {k: [round(prob, 3) for prob in probs] for k, probs in bot1_prob_crew_rescue.items()}

    print(bot1_avg_rescue_moves, bot1_prob_crew_rescue, bot1_avg_crew_saved, "\n")

    plot_Bot1(alpha_values, k_values, bot1_avg_rescue_moves, 'Average Rescue Moves', 1)
    plot_Bot1(alpha_values, k_values, bot1_prob_crew_rescue, 'Probability of Crew Rescue', 2)
    plot_Bot1(alpha_values, k_values, bot1_avg_crew_saved, 'Average Crew Saved', 3)

In [10]:
alpha_values = [0.003, 0.004, 0.005, 0.01] # 0.004 > 0.01
k_values = [3]
max_iter = 30
timeout = 10000

In [ ]:
one_alien_one_crew(alpha_values, k_values, max_iter, timeout)

Bot: (20, 4), Crew: [(2, 13)], Aliens: [(27, 19)]
Bot: (20, 5), Crew: [(2, 13)], Aliens: [(27, 19)]
Bot: (20, 6), Crew: [(2, 13)], Aliens: [(27, 19)]
Bot: (21, 6), Crew: [(2, 13)], Aliens: [(27, 19)]
Bot: (22, 6), Crew: [(2, 13)], Aliens: [(27, 19)]
Bot: (21, 6), Crew: [(2, 13)], Aliens: [(28, 19)]
Bot: (20, 6), Crew: [(2, 13)], Aliens: [(28, 18)]
Bot: (20, 7), Crew: [(2, 13)], Aliens: [(28, 17)]
Bot: (19, 7), Crew: [(2, 13)], Aliens: [(29, 17)]
Bot: (18, 7), Crew: [(2, 13)], Aliens: [(29, 17)]
Bot: (18, 8), Crew: [(2, 13)], Aliens: [(28, 17)]
Bot: (17, 8), Crew: [(2, 13)], Aliens: [(28, 17)]
Bot: (16, 8), Crew: [(2, 13)], Aliens: [(29, 17)]
Bot: (16, 9), Crew: [(2, 13)], Aliens: [(29, 18)]
Bot: (15, 9), Crew: [(2, 13)], Aliens: [(29, 17)]
Bot: (14, 9), Crew: [(2, 13)], Aliens: [(29, 17)]
Bot: (14, 10), Crew: [(2, 13)], Aliens: [(29, 16)]
Bot: (15, 10), Crew: [(2, 13)], Aliens: [(29, 16)]
Bot: (15, 11), Crew: [(2, 13)], Aliens: [(29, 17)]
Bot: (15, 12), Crew: [(2, 13)], Aliens: [(29, 1

Bot: (2, 11), Crew: [(2, 13)], Aliens: [(25, 0)]
Bot: (3, 11), Crew: [(2, 13)], Aliens: [(25, 0)]
Bot: (4, 11), Crew: [(2, 13)], Aliens: [(26, 0)]
Bot: (5, 11), Crew: [(2, 13)], Aliens: [(26, 1)]
Bot: (5, 12), Crew: [(2, 13)], Aliens: [(26, 1)]
Bot: (6, 12), Crew: [(2, 13)], Aliens: [(26, 1)]
Bot: (7, 12), Crew: [(2, 13)], Aliens: [(26, 2)]
Bot: (8, 12), Crew: [(2, 13)], Aliens: [(26, 1)]
Bot: (9, 12), Crew: [(2, 13)], Aliens: [(26, 2)]
Bot: (9, 11), Crew: [(2, 13)], Aliens: [(26, 1)]
Bot: (9, 12), Crew: [(2, 13)], Aliens: [(26, 0)]
Bot: (8, 12), Crew: [(2, 13)], Aliens: [(27, 0)]
Bot: (7, 12), Crew: [(2, 13)], Aliens: [(26, 0)]
Bot: (7, 11), Crew: [(2, 13)], Aliens: [(25, 0)]
Bot: (6, 11), Crew: [(2, 13)], Aliens: [(26, 0)]
Bot: (6, 10), Crew: [(2, 13)], Aliens: [(26, 0)]
Bot: (6, 9), Crew: [(2, 13)], Aliens: [(26, 0)]
Bot: (7, 9), Crew: [(2, 13)], Aliens: [(26, 1)]
Bot: (6, 9), Crew: [(2, 13)], Aliens: [(26, 2)]
Bot: (5, 9), Crew: [(2, 13)], Aliens: [(26, 2)]
Bot: (5, 8), Crew: [(2, 

Bot: (4, 15), Crew: [(10, 25)], Aliens: [(28, 0)]
Bot: (4, 16), Crew: [(10, 25)], Aliens: [(29, 0)]
Bot: (4, 15), Crew: [(10, 25)], Aliens: [(29, 0)]
Bot: (4, 14), Crew: [(10, 25)], Aliens: [(28, 0)]
Bot: (4, 13), Crew: [(10, 25)], Aliens: [(28, 1)]
Bot: (4, 13), Crew: [(10, 25)], Aliens: [(29, 1)]
Bot: (4, 14), Crew: [(10, 25)], Aliens: [(28, 1)]
Bot: (3, 14), Crew: [(10, 25)], Aliens: [(29, 1)]
Bot: (2, 14), Crew: [(10, 25)], Aliens: [(29, 0)]
Bot: (2, 15), Crew: [(10, 25)], Aliens: [(29, 0)]
Bot: (1, 15), Crew: [(10, 25)], Aliens: [(28, 0)]
Bot: (1, 16), Crew: [(10, 25)], Aliens: [(28, 1)]
Bot: (1, 17), Crew: [(10, 25)], Aliens: [(28, 1)]
Bot: (2, 17), Crew: [(10, 25)], Aliens: [(28, 1)]
Bot: (3, 17), Crew: [(10, 25)], Aliens: [(28, 1)]
Bot: (3, 18), Crew: [(10, 25)], Aliens: [(28, 0)]
Bot: (4, 18), Crew: [(10, 25)], Aliens: [(29, 0)]
Bot: (4, 19), Crew: [(10, 25)], Aliens: [(29, 1)]
Bot: (4, 20), Crew: [(10, 25)], Aliens: [(29, 0)]
Bot: (4, 21), Crew: [(10, 25)], Aliens: [(29, 1)]


Bot: (1, 29), Crew: [(5, 12)], Aliens: [(24, 6)]
Bot: (1, 28), Crew: [(5, 12)], Aliens: [(24, 7)]
Bot: (0, 28), Crew: [(5, 12)], Aliens: [(24, 6)]
Bot: (1, 28), Crew: [(5, 12)], Aliens: [(24, 6)]
Bot: (2, 28), Crew: [(5, 12)], Aliens: [(24, 6)]
Bot: (2, 29), Crew: [(5, 12)], Aliens: [(24, 6)]
Bot: (2, 28), Crew: [(5, 12)], Aliens: [(24, 5)]
Bot: (2, 27), Crew: [(5, 12)], Aliens: [(24, 6)]
Bot: (3, 27), Crew: [(5, 12)], Aliens: [(24, 6)]
Bot: (2, 27), Crew: [(5, 12)], Aliens: [(24, 5)]
Bot: (3, 27), Crew: [(5, 12)], Aliens: [(24, 6)]
Bot: (3, 27), Crew: [(5, 12)], Aliens: [(24, 7)]
Bot: (3, 26), Crew: [(5, 12)], Aliens: [(24, 8)]
Bot: (4, 26), Crew: [(5, 12)], Aliens: [(24, 9)]
Bot: (4, 25), Crew: [(5, 12)], Aliens: [(24, 9)]
Bot: (4, 24), Crew: [(5, 12)], Aliens: [(24, 9)]
Bot: (5, 24), Crew: [(5, 12)], Aliens: [(23, 9)]
Bot: (6, 24), Crew: [(5, 12)], Aliens: [(24, 9)]
Bot: (6, 25), Crew: [(5, 12)], Aliens: [(24, 8)]
Bot: (6, 25), Crew: [(5, 12)], Aliens: [(24, 8)]
Bot: (6, 24), Crew: 

Bot: (4, 24), Crew: [(5, 12)], Aliens: [(25, 3)]
Bot: (5, 24), Crew: [(5, 12)], Aliens: [(25, 4)]
Bot: (4, 24), Crew: [(5, 12)], Aliens: [(25, 4)]
Bot: (3, 24), Crew: [(5, 12)], Aliens: [(24, 4)]
Bot: (3, 23), Crew: [(5, 12)], Aliens: [(24, 5)]
Bot: (3, 24), Crew: [(5, 12)], Aliens: [(24, 6)]
Bot: (2, 24), Crew: [(5, 12)], Aliens: [(24, 6)]
Bot: (2, 25), Crew: [(5, 12)], Aliens: [(24, 6)]
Bot: (1, 25), Crew: [(5, 12)], Aliens: [(24, 6)]
Bot: (1, 26), Crew: [(5, 12)], Aliens: [(24, 7)]
Bot: (2, 26), Crew: [(5, 12)], Aliens: [(24, 7)]
Bot: (2, 27), Crew: [(5, 12)], Aliens: [(24, 8)]
Bot: (3, 27), Crew: [(5, 12)], Aliens: [(24, 8)]
Bot: (3, 26), Crew: [(5, 12)], Aliens: [(24, 8)]
Bot: (3, 26), Crew: [(5, 12)], Aliens: [(24, 8)]
Bot: (2, 26), Crew: [(5, 12)], Aliens: [(24, 9)]
Bot: (2, 26), Crew: [(5, 12)], Aliens: [(24, 8)]
Bot: (1, 26), Crew: [(5, 12)], Aliens: [(24, 7)]
Bot: (2, 26), Crew: [(5, 12)], Aliens: [(24, 8)]
Bot: (1, 26), Crew: [(5, 12)], Aliens: [(24, 9)]
Bot: (1, 26), Crew: 

...

Bot: (5, 23), Crew: [(28, 13)], Aliens: [(4, 25)]
Bot: (6, 23), Crew: [(28, 13)], Aliens: [(4, 26)]
Bot: (6, 23), Crew: [(28, 13)], Aliens: [(4, 26)]
Bot: (6, 24), Crew: [(28, 13)], Aliens: [(3, 26)]
Bot: (6, 24), Crew: [(28, 13)], Aliens: [(2, 26)]
Bot: (5, 24), Crew: [(28, 13)], Aliens: [(2, 27)]
Bot: (4, 24), Crew: [(28, 13)], Aliens: [(2, 28)]
Bot: (3, 24), Crew: [(28, 13)], Aliens: [(1, 28)]
Bot: (3, 23), Crew: [(28, 13)], Aliens: [(1, 27)]
Bot: (2, 23), Crew: [(28, 13)], Aliens: [(1, 28)]
Bot: (2, 22), Crew: [(28, 13)], Aliens: [(1, 29)]
Bot: (1, 22), Crew: [(28, 13)], Aliens: [(1, 29)]
Bot: (1, 21), Crew: [(28, 13)], Aliens: [(1, 28)]
Bot: (1, 21), Crew: [(28, 13)], Aliens: [(1, 29)]
Bot: (1, 20), Crew: [(28, 13)], Aliens: [(2, 29)]
Bot: (2, 20), Crew: [(28, 13)], Aliens: [(2, 28)]
Bot: (2, 19), Crew: [(28, 13)], Aliens: [(2, 28)]
Bot: (2, 19), Crew: [(28, 13)], Aliens: [(2, 29)]
Bot: (2, 20), Crew: [(28, 13)], Aliens: [(1, 29)]
Bot: (2, 20), Crew: [(28, 13)], Aliens: [(1, 29)]


Bot: (2, 2), Crew: [(28, 13)], Aliens: [(10, 20)]
Bot: (1, 2), Crew: [(28, 13)], Aliens: [(11, 20)]
Bot: (1, 3), Crew: [(28, 13)], Aliens: [(10, 20)]
Bot: (1, 3), Crew: [(28, 13)], Aliens: [(9, 20)]
Bot: (1, 2), Crew: [(28, 13)], Aliens: [(8, 20)]
Bot: (1, 2), Crew: [(28, 13)], Aliens: [(8, 19)]
Bot: (1, 3), Crew: [(28, 13)], Aliens: [(8, 19)]
Bot: (1, 2), Crew: [(28, 13)], Aliens: [(8, 19)]
Bot: (1, 2), Crew: [(28, 13)], Aliens: [(8, 20)]
Bot: (1, 3), Crew: [(28, 13)], Aliens: [(9, 20)]
Bot: (1, 2), Crew: [(28, 13)], Aliens: [(8, 20)]
Bot: (1, 2), Crew: [(28, 13)], Aliens: [(9, 20)]
Bot: (1, 3), Crew: [(28, 13)], Aliens: [(9, 20)]
Bot: (1, 3), Crew: [(28, 13)], Aliens: [(9, 20)]
Bot: (1, 3), Crew: [(28, 13)], Aliens: [(8, 20)]
Bot: (1, 3), Crew: [(28, 13)], Aliens: [(7, 20)]
Bot: (1, 2), Crew: [(28, 13)], Aliens: [(8, 20)]
Bot: (1, 3), Crew: [(28, 13)], Aliens: [(8, 19)]
Bot: (1, 2), Crew: [(28, 13)], Aliens: [(8, 19)]
Bot: (0, 2), Crew: [(28, 13)], Aliens: [(8, 20)]
Bot: (0, 1), Crew

Bot: (27, 19), Crew: [(28, 13)], Aliens: [(1, 24)]
Bot: (28, 19), Crew: [(28, 13)], Aliens: [(1, 24)]
Bot: (29, 19), Crew: [(28, 13)], Aliens: [(0, 24)]
Bot: (29, 18), Crew: [(28, 13)], Aliens: [(1, 24)]
Bot: (29, 17), Crew: [(28, 13)], Aliens: [(1, 25)]
Bot: (29, 16), Crew: [(28, 13)], Aliens: [(1, 25)]
Bot: (29, 15), Crew: [(28, 13)], Aliens: [(1, 24)]
Bot: (28, 15), Crew: [(28, 13)], Aliens: [(2, 24)]
Bot: (28, 14), Crew: [(28, 13)], Aliens: [(1, 24)]
Crew saved! Win Count: 20, Loss Count: 6
Bot: (28, 13), Crew: [(11, 8)], Aliens: [(2, 24)]
Bot: (28, 12), Crew: [(11, 8)], Aliens: [(2, 23)]
Bot: (28, 11), Crew: [(11, 8)], Aliens: [(2, 23)]
Bot: (28, 10), Crew: [(11, 8)], Aliens: [(2, 24)]
Bot: (29, 10), Crew: [(11, 8)], Aliens: [(3, 24)]
Bot: (29, 9), Crew: [(11, 8)], Aliens: [(2, 24)]
Bot: (29, 8), Crew: [(11, 8)], Aliens: [(2, 24)]
Bot: (28, 8), Crew: [(11, 8)], Aliens: [(2, 25)]
Bot: (28, 8), Crew: [(11, 8)], Aliens: [(1, 25)]
Bot: (29, 8), Crew: [(11, 8)], Aliens: [(1, 26)]
Bot: 

Bot: (3, 29), Crew: [(11, 8)], Aliens: [(0, 24)]
Bot: (4, 29), Crew: [(11, 8)], Aliens: [(0, 23)]
Bot: (4, 28), Crew: [(11, 8)], Aliens: [(0, 22)]
Bot: (5, 28), Crew: [(11, 8)], Aliens: [(0, 23)]
Bot: (4, 28), Crew: [(11, 8)], Aliens: [(0, 24)]
Bot: (5, 28), Crew: [(11, 8)], Aliens: [(0, 23)]
Bot: (5, 27), Crew: [(11, 8)], Aliens: [(0, 22)]
Bot: (5, 27), Crew: [(11, 8)], Aliens: [(0, 23)]
Bot: (5, 27), Crew: [(11, 8)], Aliens: [(0, 24)]
Bot: (5, 28), Crew: [(11, 8)], Aliens: [(0, 23)]
Bot: (4, 28), Crew: [(11, 8)], Aliens: [(0, 23)]
Bot: (4, 28), Crew: [(11, 8)], Aliens: [(0, 22)]
Bot: (4, 28), Crew: [(11, 8)], Aliens: [(0, 23)]
Bot: (4, 28), Crew: [(11, 8)], Aliens: [(0, 22)]
Bot: (5, 28), Crew: [(11, 8)], Aliens: [(0, 23)]
Bot: (4, 28), Crew: [(11, 8)], Aliens: [(0, 24)]
Bot: (5, 28), Crew: [(11, 8)], Aliens: [(0, 24)]
Bot: (4, 28), Crew: [(11, 8)], Aliens: [(1, 24)]
Bot: (4, 29), Crew: [(11, 8)], Aliens: [(1, 25)]
Bot: (4, 28), Crew: [(11, 8)], Aliens: [(1, 24)]
Bot: (4, 29), Crew: 

Bot: (1, 0), Crew: [(11, 8)], Aliens: [(2, 28)]
Bot: (1, 0), Crew: [(11, 8)], Aliens: [(2, 28)]
Bot: (2, 0), Crew: [(11, 8)], Aliens: [(2, 28)]
Bot: (1, 0), Crew: [(11, 8)], Aliens: [(1, 28)]
Bot: (0, 0), Crew: [(11, 8)], Aliens: [(1, 29)]
Bot: (0, 1), Crew: [(11, 8)], Aliens: [(1, 29)]
Bot: (0, 1), Crew: [(11, 8)], Aliens: [(1, 29)]
Bot: (0, 1), Crew: [(11, 8)], Aliens: [(1, 29)]
Bot: (0, 1), Crew: [(11, 8)], Aliens: [(1, 28)]
Bot: (0, 0), Crew: [(11, 8)], Aliens: [(1, 28)]
Bot: (0, 1), Crew: [(11, 8)], Aliens: [(0, 28)]
Bot: (0, 2), Crew: [(11, 8)], Aliens: [(1, 28)]
Bot: (0, 1), Crew: [(11, 8)], Aliens: [(1, 28)]
Bot: (0, 2), Crew: [(11, 8)], Aliens: [(0, 28)]
Bot: (1, 2), Crew: [(11, 8)], Aliens: [(0, 28)]
Bot: (2, 2), Crew: [(11, 8)], Aliens: [(0, 28)]
Bot: (2, 2), Crew: [(11, 8)], Aliens: [(1, 28)]
Bot: (1, 2), Crew: [(11, 8)], Aliens: [(1, 29)]
Bot: (2, 2), Crew: [(11, 8)], Aliens: [(2, 29)]
Bot: (2, 2), Crew: [(11, 8)], Aliens: [(2, 28)]
Bot: (2, 2), Crew: [(11, 8)], Aliens: [(